In [1]:
import pandas as pd
import numpy as np
import scipy
import os
from tqdm.notebook import tqdm
from time import time
from collections import defaultdict
from scipy.sparse import csr_matrix,save_npz
import sys

In [2]:
# Get tfidf file count
file_count = max([int(x[6:8].replace(".","")) for x in os.listdir('tfidf') if '.' in x])+1

In [3]:
# count the freq of doc having specific words
have_word_count_dict = defaultdict(int)
tot_doc_cnt = 0

for i in tqdm(range(file_count)):
    single_file = np.load('tfidf/value_%d.npy'%i,allow_pickle=True)
    for doc in single_file:
        for wrd in doc:
            have_word_count_dict[wrd] += 1
    tot_doc_cnt += len(single_file)

  0%|          | 0/87 [00:00<?, ?it/s]

In [4]:
# Create dictionary of words:index from frequency dictionary
wrd_idx_map = {wrd:np.int32(idx) for idx,wrd in enumerate(have_word_count_dict)}

In [5]:
# Update dictionary for each word with log of (total document count/count of word)
for wrd, wrd_cnt in have_word_count_dict.items():
    have_word_count_dict[wrd] = np.log(tot_doc_cnt/wrd_cnt)

In [11]:
indice_list = []
value_list = []
dptr_list = [0]

for i in tqdm(range(file_count)):
    # Load npy file
    single_file = np.load('tfidf/value_%d.npy'%i,allow_pickle=True)
    
    for doc_idx, doc in enumerate(single_file):
        # Get words list from doc
        wrd_list = [wrd for wrd in doc.keys()]
        # Get words count from doc
        wrd_cnt_list = list(doc.values())
        # Multiply element-wise normalize word counts by their log of (total document count/count of word)
        wrd_cnt_list = np.multiply(
            np.divide(wrd_cnt_list, 
                      np.sum(wrd_cnt_list)
                     ),
            [have_word_count_dict.get(wrd) for wrd in wrd_list]
        )
        
        # Add to end of indices, value, and dptr lists
        indice_list.extend([wrd_idx_map.get(wrd) for wrd in wrd_list])
        value_list.extend(wrd_cnt_list)
        dptr_list.append(dptr_list[-1]+len(wrd_cnt_list))
    
# Save result to npz file
    if (i+1) % 10 == 0:
        result = csr_matrix((value_list, indice_list, dptr_list))
        scipy.sparse.save_npz('tfidf/sparse_tfidf_mat/tfidf_%d.npz'%(i // 10), result)
        indice_list = []
        value_list = []
        dptr_list = [0]
        
result = csr_matrix((value_list, indice_list, dptr_list))
scipy.sparse.save_npz('tfidf/sparse_tfidf_mat/tfidf_%d.npz'%(i // 10), result)

  0%|          | 0/87 [00:00<?, ?it/s]

In [25]:
# Save to npy file
index_list = []

for i in tqdm(range(file_count)):
    single_file = np.load('tfidf/index_%d.npy'%i,allow_pickle=True)
    index_list.extend(single_file)
        
    if (i+1) % 10 == 0:
        np.save('tfidf/sparse_tfidf_mat/index_%d.npy'%(i // 10), index_list)
        index_list = []
        
np.save('tfidf/sparse_tfidf_mat/index_%d.npy'%(i // 10), index_list)

  0%|          | 0/87 [00:00<?, ?it/s]